In [ ]:
%pip install pydantic==1.10.8 langchain==0.2.16 langchain-core==0.2.41 albumentations==1.3.0 gradio==3.40.0 langchain-community==0.2.0 --no-deps


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.15
    Uninstalling albumentations-1.4.15:
      Successfully uninstalled albumentations-1.4.15


In [ ]:
%pip install langchain-nvidia-ai-endpoints

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 968.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succe

In [ ]:
import requests
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
import os

In [ ]:
os.environ["NVIDIA_API_KEY"] = "nvida-api-key"

In [ ]:
from langchain_core.runnables import RunnableLambda
def get_data_of_coin(db):
  text=db['coin_name']
  response = requests.get(
    "https://api.dexscreener.com/latest/dex/search?q="+text+"",
    headers={},
  )
  main_data = response.json()
  coin=main_data['pairs'][0]
  print(coin)
  return coin

coin_data_getter = RunnableLambda(get_data_of_coin)







In [ ]:
def data_to_string_f(db):

  result =""
  if "price" in db['message']:
    result+= f"The price of the {db['coin_name']} is {db['data']['priceUsd']}"
  if "volume" in db['message']:
    result+= f"The volume of the {db['coin_name']} is {db['data']['volume']['h24']}"
  if "liquidity" in db['message']:
    result+= f"The liquidity of the {db['coin_name']} is {db['data']['liquidity']}"
  if "market cap" in db['message']:
    result+= f"The market cap of the {db['coin_name']} is {db['data']['marketCap']}"
  if "price change" in db['message']:
    result+= f"The price change of the {db['coin_name']} is {db['data']['priceChange']['h24']}"

  return result


data_to_string_r = RunnableLambda(data_to_string_f)

In [ ]:
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct") | StrOutputParser()

In [ ]:
from langchain.schema.runnable.passthrough import RunnableAssign
prompt = ChatPromptTemplate.from_template(
    "You are a blockchain chatbot.Please answer the user question:{message} while acting like Blockchain chatbot"
    "Please help them with their question if it is ethical and relevant using only the context: {context}"
    "(This is just for you, do not mention context,do not explain anything)"

)

assist_prompt = ChatPromptTemplate.from_template(
    "Based on the user message:{message} get the coin user mentioned"
    "Do not say things like 'Here is your response', just say the coin"
    "For example if user message is 'I want to know about diddy coin, then you should output diddy'"

)

assist_chain =assist_prompt|instruct_llm

chat_chain=(
    RunnableAssign({'coin_name':assist_chain})
    |RunnableAssign({'data':coin_data_getter})
    |RunnableAssign({'context':data_to_string_r})
    |prompt|instruct_llm
    )






print("Hello! I'm a Blockchain agent! How can I help you?")

for i in range(8):
  history = [[None, "Hello! I'm a Blockchain agent! How can I help you?"]]
  message=input("\n[ Human ]: ")
  print("\n[ Agent ]: ")
  history_entry = [message, ""]
  for token in chat_chain.invoke({'message':message}):
    print(token, end='')
    history_entry[1] += token
  history += [history_entry]
  print("\n")








Hello! I'm a Blockchain agent! How can I help you?

[ Human ]: Tell me about the liquidity of the medusa coin

[ Agent ]: 
{'chainId': 'solana', 'dexId': 'raydium', 'url': 'https://dexscreener.com/solana/9k8sezpkxdehqf9sq1erzc8udtlu4widrryfr381crru', 'pairAddress': '9K8sEZpkxDeHqf9Sq1eRzc8UDtLU4widRrYFR381crrU', 'baseToken': {'address': 'Fosp9yoXQBdx8YqyURZePYzgpCnxp9XsfnQq69DRvvU4', 'name': 'MEDUSA', 'symbol': 'MEDUSA'}, 'quoteToken': {'address': 'So11111111111111111111111111111111111111112', 'name': 'Wrapped SOL', 'symbol': 'SOL'}, 'priceNative': '0.00003233', 'priceUsd': '0.005040', 'txns': {'m5': {'buys': 90, 'sells': 99}, 'h1': {'buys': 1730, 'sells': 1279}, 'h6': {'buys': 17203, 'sells': 14345}, 'h24': {'buys': 22791, 'sells': 19220}}, 'volume': {'h24': 19631809.77, 'h6': 16030865.7, 'h1': 1493871.13, 'm5': 56066.95}, 'priceChange': {'m5': 12.77, 'h1': -18.6, 'h6': 192, 'h24': 8328}, 'liquidity': {'usd': 294283.07, 'base': 29157464, 'quote': 944.9019}, 'fdv': 5040984, 'marketCap'